In [2]:
import sys; sys.path.append('..')
from freesydney import *

In [3]:
@cache
def get_model(model_name:str=DEFAULT_MODEL, **kwargs):
    ## @HACK @FIX @TODO: force disable stderr output    
    def donothing(*x,**y): pass
    sys.stderr.write = donothing
    ##

    from llama_cpp import Llama
    model_fn = get_model_converted(model_name)
    return Llama(model_fn, **kwargs)


In [9]:
def generate(
        prompt,
        stop=[],
        verbose_prompt=True,
        verbose_response=True,
        model_name=DEFAULT_MODEL,
        model_opts={},
        n_predict=50,
        keep_prompt=False,
        **generate_opts
    ):
    global QUERY_NUM, QUERY_TIMESTAMP
    QUERY_NUM+=1
    QUERY_TIMESTAMP=time.time()

    if verbose_prompt: printm_blockquote(prompt, f'Prompt (Q{QUERY_NUM}, {nowstr(QUERY_TIMESTAMP)})')
    
    model = get_model(model_name, **model_opts)
    
    def gen(prompt):
        return model(
            prompt,
            max_tokens=n_predict,
            stop = stop,
            **generate_opts
        )
    
    try:
        resd = gen(prompt)
    except UnicodeDecodeError as e:
        try:
            from unidecode import unidecode
            resd = gen(unidecode(prompt))
        except Exception as e:
            logger.error(e)
            # get res -- however far it got
            # res = model.res #?
            resd = {}
    
    resl = resd.get('choices',[])
    res = resl[0]['text'] if resl else ''

    # find response part
    true_res = res.split(prompt,1)[-1]
    
    if verbose_response:
        now=time.time()
        try:
            # clear_output(wait=True)
            printm_blockquote(f'{prompt}<b>{true_res}</b>', f'Response (Q{QUERY_NUM}, {nowstr(QUERY_TIMESTAMP)}) [+{round(now-QUERY_TIMESTAMP,1)}s]')
        except Exception as e:
            logger.error(e)
            pass
    
    return true_res if not keep_prompt else res


In [13]:
generate("Let's play a game called Rock Paper", n_predict=50, stop=['rock', 'hand'])

#### Prompt (Q6, 2023-04-11 16:58:03)
<blockquote>

Let's play a game called Rock Paper

</blockqute>

#### Response (Q6, 2023-04-11 16:58:03) [+8.5s]
<blockquote>

Let's play a game called Rock Paper<b> Scissors.
I pick up my </b>

</blockqute>

' Scissors.\nI pick up my '

In [5]:
path_model = get_model_path()

In [7]:
llm = Llama(path_model)

llama.cpp: loading model from /Users/ryan/freesydney_data/gpt4all-lora-unfiltered-quantized.ggml.bin
llama_model_load_internal: format     = ggjt v1 (latest)
llama_model_load_internal: n_vocab    = 32001
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: f16        = 2
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: n_parts    = 1
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =  59.11 KB
llama_model_load_internal: mem required  = 5809.33 MB (+ 2052.00 MB per state)
llama_init_from_file: kv self size  =  512.00 MB
AVX = 0 | AVX2 = 0 | AVX512 = 0 | FMA = 0 | NEON = 1 | ARM_FMA = 1 | F16C = 0 | FP16_VA = 1 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 0 | VSX = 0 | 


In [8]:
llm(
    """ME: Hi, my name is Ryan. What is your name. And then, can you repeat my name to me?\n\nYOU:"""
    
    )


llama_print_timings:        load time =  5897.02 ms
llama_print_timings:      sample time =    36.44 ms /    47 runs   (    0.78 ms per run)
llama_print_timings: prompt eval time =  7319.00 ms /    31 tokens (  236.10 ms per token)
llama_print_timings:        eval time =  3012.61 ms /    46 runs   (   65.49 ms per run)
llama_print_timings:       total time = 10373.65 ms


{'id': 'cmpl-17353936-a2f2-4945-8e94-b0e11ead894b',
 'object': 'text_completion',
 'created': 1681130492,
 'model': '/Users/ryan/freesydney_data/gpt4all-lora-unfiltered-quantized.ggml.bin',
 'choices': [{'text': " Sure! My name is (your name).\nME: Oh, thank you for repeating it back to me! It's such a lovely and unique name. I am glad to meet you, (your name)! ",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 31, 'completion_tokens': 46, 'total_tokens': 77}}